In [42]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [43]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [44]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [45]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [46]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [47]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

print(len(input_texts))

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1
2154


In [48]:
input_texts[:5]

['lo death hath reard himself a throne',
 'in a strange city all alone',
 'far down within the dim west',
 'where the good and the bad and the worst and the best',
 'have gone to their eternal rest']

In [49]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels, test_size=0.2, random_state=42)

In [50]:
len(Ytrain), len(Ytest)

(1723, 431)

In [51]:
train_text[:5]

['and many must have seen him make',
 'with flowers in her lap until they fade',
 'what signify a donkeys cars and bottle',
 'son mother can make a common table rear',
 'if the wish hadnt been too strong in me']

In [52]:
Ytrain[:5]

[1, 1, 1, 1, 1]

In [53]:
idx = 1
word2idx = {'<unk>': 0}

In [54]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [55]:
word2idx

{'<unk>': 0,
 'and': 1,
 'many': 2,
 'must': 3,
 'have': 4,
 'seen': 5,
 'him': 6,
 'make': 7,
 'with': 8,
 'flowers': 9,
 'in': 10,
 'her': 11,
 'lap': 12,
 'until': 13,
 'they': 14,
 'fade': 15,
 'what': 16,
 'signify': 17,
 'a': 18,
 'donkeys': 19,
 'cars': 20,
 'bottle': 21,
 'son': 22,
 'mother': 23,
 'can': 24,
 'common': 25,
 'table': 26,
 'rear': 27,
 'if': 28,
 'the': 29,
 'wish': 30,
 'hadnt': 31,
 'been': 32,
 'too': 33,
 'strong': 34,
 'me': 35,
 'town': 36,
 'that': 37,
 'is': 38,
 'no': 39,
 'more': 40,
 'on': 41,
 'thine': 42,
 'own': 43,
 'upturnd': 44,
 'alas': 45,
 'sorrow': 46,
 'heavy': 47,
 'iron': 48,
 'bells': 49,
 'front': 50,
 'of': 51,
 'one': 52,
 'hall': 53,
 'id': 54,
 'left': 55,
 'hitched': 56,
 'single': 57,
 'leaf': 58,
 'was': 59,
 'bough': 60,
 'unembodied': 61,
 'essence': 62,
 'hiss': 63,
 'spray': 64,
 'it': 65,
 'will': 66,
 'go': 67,
 'hard': 68,
 'john': 69,
 'he': 70,
 'do': 71,
 'you': 72,
 'not': 73,
 'mind': 74,
 'certain': 75,
 'coolness': 

In [56]:
len(word2idx)

2612

In [57]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [58]:
train_text_int[100:105]

[[73, 388, 389, 41, 390, 1, 391],
 [8, 392, 283, 157, 393],
 [1, 80, 352, 394, 38, 395, 396, 397],
 [29, 398, 399],
 [400, 394, 401, 402, 283, 258, 35, 403, 72]]

In [59]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [60]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [61]:
train_text_int

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 1, 21],
 [22, 23, 24, 7, 18, 25, 26, 27],
 [28, 29, 30, 31, 32, 33, 34, 10, 35],
 [1, 10, 18, 36, 37, 38, 39, 40, 18, 36],
 [1, 41, 42, 43, 44, 45, 10, 46],
 [29, 47, 48, 49],
 [10, 50, 51, 52, 36, 53, 54, 55, 6, 56],
 [18, 57, 58, 59, 41, 18, 60],
 [29, 61, 62, 1, 39, 40],
 [8, 29, 63, 51, 64],
 [65, 66, 67, 68, 8, 69, 16, 66, 70, 71],
 [72, 3, 73, 74, 18, 75, 76, 77, 6],
 [78, 79, 78, 79, 80, 81, 1, 82],
 [83, 84, 85, 84, 86],
 [87, 88, 29, 89],
 [80, 90, 6, 71, 65, 91, 35, 10, 29, 92],
 [69, 53, 93, 35, 73, 28, 70, 94, 95, 96],
 [97, 98, 73, 29, 99, 100, 51, 101],
 [37, 102, 103, 10, 104, 91, 29, 105],
 [106, 107, 51, 108, 29, 109, 110, 111, 112],
 [113, 37, 114, 51, 115],
 [18, 116, 91, 18, 117, 118, 119, 118, 120],
 [37, 121, 122],
 [10, 29, 123, 51, 124, 125],
 [126, 80, 127, 103, 51, 128, 129],
 [130, 131, 132, 80, 133, 134, 6, 135],
 [80, 136, 80, 137, 138, 51, 139],
 [140, 141, 142, 29, 143, 51, 29,

In [62]:
pi0.sum()

3182.0

In [63]:
A0.sum()

6825772.0

In [64]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [65]:
pi0

array([0.00031427, 0.01539912, 0.00031427, ..., 0.00031427, 0.00031427,
       0.00031427])

In [66]:
A0

array([[0.00038285, 0.00038285, 0.00038285, ..., 0.00038285, 0.00038285,
        0.00038285],
       [0.0003667 , 0.0003667 , 0.0003667 , ..., 0.0003667 , 0.0003667 ,
        0.0003667 ],
       [0.00038183, 0.00038183, 0.00038183, ..., 0.00038183, 0.00038183,
        0.00038183],
       ...,
       [0.00038285, 0.00038285, 0.00038285, ..., 0.00038285, 0.00038285,
        0.00038285],
       [0.00038285, 0.00038285, 0.00038285, ..., 0.00038285, 0.00038285,
        0.00038285],
       [0.00038285, 0.00038285, 0.00038285, ..., 0.00038285, 0.00038285,
        0.00038285]])

In [67]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [68]:
logpi0

array([-8.06526521, -4.17344491, -8.06526521, ..., -8.06526521,
       -8.06526521, -8.06526521])

In [69]:
logA0

array([[-7.86787149, -7.86787149, -7.86787149, ..., -7.86787149,
        -7.86787149, -7.86787149],
       [-7.91095738, -7.91095738, -7.91095738, ..., -7.91095738,
        -7.91095738, -7.91095738],
       [-7.87054784, -7.87054784, -7.87054784, ..., -7.87054784,
        -7.87054784, -7.87054784],
       ...,
       [-7.86787149, -7.86787149, -7.86787149, ..., -7.86787149,
        -7.86787149, -7.86787149],
       [-7.86787149, -7.86787149, -7.86787149, ..., -7.86787149,
        -7.86787149, -7.86787149],
       [-7.86787149, -7.86787149, -7.86787149, ..., -7.86787149,
        -7.86787149, -7.86787149]])

In [70]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3308183401044689, 0.669181659895531)

In [71]:
(logp0,logp1)

(-1.106185875699334, -0.40169971625887063)

In [72]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [73]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [74]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.995937318630296


In [75]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8445475638051044


In [76]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [77]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 563,    7],
       [   0, 1153]])

In [78]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 85,  63],
       [  4, 279]])

In [79]:
f1_score(Ytrain, Ptrain)

0.9969736273238219

In [80]:
f1_score(Ytest, Ptest)

0.8928